EDGE DETECTION


In [1]:
# Define the image array (2D list) representing a grayscale image
image = [
    [10, 10, 10, 10, 10],
    [10, 100, 100, 100, 10],
    [10, 100, 100, 100, 10],
    [10, 100, 100, 100, 10],
    [10, 10, 10, 10, 10]
]

# Define vertical and horizontal edge detection kernels (Sobel filters)
vertical_kernel = [
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
]

horizontal_kernel = [
    [-1, -2, -1],
    [0,  0,  0],
    [1,  2,  1]
]

# Function to perform convolution
def convolve(image, kernel):
    rows = len(image)
    cols = len(image[0])
    k_size = len(kernel)
    k_half = k_size // 2
    
    # Initialize the output array with zeros
    output = [[0 for _ in range(cols)] for _ in range(rows)]
    
    # Perform convolution
    for i in range(k_half, rows - k_half):
        for j in range(k_half, cols - k_half):
            weighted_sum = 0
            for ki in range(k_size):
                for kj in range(k_size):
                    pixel = image[i + ki - k_half][j + kj - k_half]
                    weight = kernel[ki][kj]
                    weighted_sum += pixel * weight
            output[i][j] = abs(weighted_sum)  # Taking absolute value for edge intensity
    return output

# Apply vertical and horizontal kernels
vertical_edges = convolve(image, vertical_kernel)
horizontal_edges = convolve(image, horizontal_kernel)

# Display the resulting arrays
print("Original Image:")
for row in image:
    print(row)

print("\nVertical Edges Detected:")
for row in vertical_edges:
    print(row)

print("\nHorizontal Edges Detected:")
for row in horizontal_edges:
    print(row)


Original Image:
[10, 10, 10, 10, 10]
[10, 100, 100, 100, 10]
[10, 100, 100, 100, 10]
[10, 100, 100, 100, 10]
[10, 10, 10, 10, 10]

Vertical Edges Detected:
[0, 0, 0, 0, 0]
[0, 270, 0, 270, 0]
[0, 360, 0, 360, 0]
[0, 270, 0, 270, 0]
[0, 0, 0, 0, 0]

Horizontal Edges Detected:
[0, 0, 0, 0, 0]
[0, 270, 360, 270, 0]
[0, 0, 0, 0, 0]
[0, 270, 360, 270, 0]
[0, 0, 0, 0, 0]


LENET-5 BASICS

LeNet-5 Architecture
Input Layer:

Accepts grayscale images of size 32x32 pixels.
MNIST images (28x28) are resized to fit.
Convolutional Layer 1 (C1):

6 filters of size 5x5 applied with a stride of 1.
Output size: 28x28x6 (6 feature maps).
Subsampling Layer 1 (S2):

Average pooling with a 2x2 kernel and stride of 2.
Output size: 14x14x6 (reduces spatial dimensions).
Convolutional Layer 2 (C3):

16 filters of size 5x5, connected selectively to previous layer maps.
Output size: 10x10x16.
Subsampling Layer 2 (S4):

Another average pooling layer with a 2x2 kernel.
Output size: 5x5x16.
Fully Connected Layer 1 (F5):

Input from previous layer flattened to 1D.
120 neurons.
Fully Connected Layer 2 (Output Layer):

84 neurons in a dense layer, connected to 10 output neurons (one per digit class).
Key Point
Uses tanh activation instead of ReLU.


In [3]:
def build_lenet(input_shape):
  # Define Sequential Model
  model = tf.keras.Sequential()
  
  # C1 Convolution Layer
  model.add(tf.keras.layers.Conv2D(filters=6, strides=(1,1), kernel_size=(5,5), activation='tanh', input_shape=input_shape))
  
  # S2 SubSampling Layer
  model.add(tf.keras.layers.AveragePooling2D(pool_size=(2,2), strides=(2,2)))

  # C3 Convolution Layer
  model.add(tf.keras.layers.Conv2D(filters=6, strides=(1,1), kernel_size=(5,5), activation='tanh'))

  # S4 SubSampling Layer
  model.add(tf.keras.layers.AveragePooling2D(pool_size=(2,2), strides=(2,2)))

  # C5 Fully Connected Layer
  model.add(tf.keras.layers.Dense(units=120, activation='tanh'))

  # Flatten the output so that we can connect it with the fully connected layers by converting it into a 1D Array
  model.add(tf.keras.layers.Flatten())

  # FC6 Fully Connected Layers
  model.add(tf.keras.layers.Dense(units=84, activation='tanh'))

  # Output Layer
  model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

  # Compile the Model
  model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(lr=0.1, momentum=0.0, decay=0.0), metrics=['accuracy'])

  return model

CLASSIFICATION REPORT

In [ ]:
def calculate_metrics(actual_labels, predicted_labels, num_classes=10):
    # Initialize counts
    true_positive = [0] * num_classes
    false_positive = [0] * num_classes
    false_negative = [0] * num_classes

    # Populate TP, FP, FN for each class
    for i in range(len(actual_labels)):
        true_label = int(actual_labels[i])
        pred_label = int(predicted_labels[i])
        if true_label == pred_label:
            true_positive[true_label] += 1
        else:
            false_positive[pred_label] += 1
            false_negative[true_label] += 1

    # Calculate precision, recall, and F1-score for each class
    precision = []
    recall = []
    f1_score = []

    for i in range(num_classes):
        tp = true_positive[i]
        fp = false_positive[i]
        fn = false_negative[i]

        # Precision: TP / (TP + FP)
        p = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        precision.append(p)

        # Recall: TP / (TP + FN)
        r = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        recall.append(r)

        # F1-Score: 2 * (Precision * Recall) / (Precision + Recall)
        f1 = 2 * (p * r) / (p + r) if (p + r) > 0 else 0.0
        f1_score.append(f1)

    # Calculate macro-average
    macro_precision = sum(precision) / num_classes
    macro_recall = sum(recall) / num_classes
    macro_f1 = sum(f1_score) / num_classes

    # Print results
    print("Class\tPrecision\tRecall\t\tF1-Score")
    for i in range(num_classes):
        print(f"{i}\t{precision[i]:.2f}\t\t{recall[i]:.2f}\t\t{f1_score[i]:.2f}")

    print("\nMacro-Average Metrics:")
    print(f"Precision: {macro_precision:.2f}")
    print(f"Recall: {macro_recall:.2f}")
    print(f"F1-Score: {macro_f1:.2f}")

    return precision, recall, f1_score, macro_precision, macro_recall, macro_f1




# Assuming act_labels and model_predicted are tensors from the test phase
precision, recall, f1_score, macro_precision, macro_recall, macro_f1 = calculate_metrics(
    act_labels.numpy(), model_predicted.numpy(), num_classes=10
)


CNN

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np


In [6]:
def build_cnn():
    model = Sequential([
        # Convolutional layer
        Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        
        # Another convolutional layer
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        
        # Flattening
        Flatten(),
        
        # Fully connected layers
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(2, activation='softmax')  # Output layer with 2 classes
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'AutismDataset/train',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    'AutismDataset/test',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)
